# 🚀 Pipeline Completo: Análisis de Imágenes CT Pulmonares (LUNA16)

**Notebook Principal** - Visión general del pipeline completo de procesamiento

Este notebook es el punto de entrada principal al proyecto. Muestra el flujo completo desde la carga de datos hasta la visualización de resultados.

---

## 📋 Contenido del Pipeline

1. **Configuración del entorno** (local o Google Colab)
2. **Carga de datos LUNA16** (archivos .mhd/.raw)
3. **Preprocesamiento** (segmentación pulmonar, normalización)
4. **Visualización** (slices, anotaciones, máscaras)
5. **Análisis de nódulos** (detección y caracterización)
6. **Resultados y métricas**

---

## ⚙️ Configuración del Entorno

In [ ]:
# Detectar entorno de ejecución
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔵 Ejecutando en Google Colab")
    print("="*50)
    
    # Montar Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Instalar dependencias
    print("\n📦 Instalando dependencias...")
    import subprocess
    paquetes = ['SimpleITK', 'scikit-image']
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + paquetes)
    
    # Configurar path al código utils/
    # Opción 1: Si clonaste el repositorio
    # !git clone https://github.com/tu-usuario/tu-repo.git
    # sys.path.append('/content/tu-repo')
    
    # Opción 2: Si tienes los archivos en Google Drive
    # sys.path.append('/content/drive/MyDrive/Imagenes Biomedicas')
    
    print("✅ Configuración de Colab completada\n")
    
else:
    print("🟢 Ejecutando localmente")
    print("="*50)
    
    # Añadir directorio padre al path para importar utils
    parent_dir = os.path.abspath('..')
    if parent_dir not in sys.path:
        sys.path.insert(0, parent_dir)
    
    print(f"📂 Directorio de trabajo: {os.getcwd()}")
    print("✅ Configuración local completada\n")

## 📚 Importar Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# Importar nuestros módulos personalizados
from utils import LUNA16DataLoader, LungPreprocessor, LungVisualizer, SegmentationMetrics

# Configurar matplotlib
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100

print("✅ Librerías importadas correctamente")
print("\n📦 Módulos disponibles:")
print("  - LUNA16DataLoader: Carga de datos .mhd/.raw")
print("  - LungPreprocessor: Segmentación y preprocesamiento")
print("  - LungVisualizer: Visualización de imágenes CT")
print("  - SegmentationMetrics: Métricas de evaluación")

## 🗂️ Configuración de Rutas a los Datos

**IMPORTANTE**: Ajusta estas rutas según tu configuración:

In [ ]:
import os

if IN_COLAB:
    # ===== CONFIGURACIÓN PARA GOOGLE COLAB =====
    # Rutas en Google Drive (MODIFICA según tu estructura en Drive)
    DATA_PATH = '/content/drive/MyDrive/LUNA16/subset0'
    ANNOTATIONS_PATH = '/content/drive/MyDrive/LUNA16/annotations.csv'
    
else:
    # ===== CONFIGURACIÓN PARA EJECUCIÓN LOCAL =====
    
    # Buscar LUNA16 en el directorio padre de notebooks/
    # Estructura esperada:
    #   Imagenes Biomedicas/
    #   ├── notebooks/        <- estás aquí
    #   ├── LUNA16/
    #   │   ├── subset0/
    #   │   │   └── *.mhd, *.raw
    #   │   └── annotations.csv
    #   └── utils/
    
    project_root = os.path.abspath('..')  # Directorio padre de notebooks/
    
    possible_paths = [
        os.path.join(project_root, 'LUNA16', 'subset0'),  # Recomendado
        os.path.join(project_root, 'data', 'LUNA16', 'subset0'),
        os.path.join(project_root, 'data', 'subset0'),
        os.path.join(os.path.expanduser('~'), 'Desktop', 'LUNA16', 'subset0'),
        os.path.join(os.path.expanduser('~'), 'Documents', 'LUNA16', 'subset0'),
    ]
    
    DATA_PATH = None
    for path in possible_paths:
        if os.path.exists(path):
            DATA_PATH = path
            print(f"✅ Datos encontrados en: {DATA_PATH}")
            break
    
    # Si no se encontró, pedir al usuario
    if DATA_PATH is None:
        print("⚠️  No se encontraron datos LUNA16.")
        print("\n🔍 Ubicaciones buscadas:")
        for p in possible_paths:
            print(f"  - {p}")
        print("\n💡 Recomendación: Coloca la carpeta LUNA16 en el directorio raíz del proyecto:")
        print(f"   {os.path.join(project_root, 'LUNA16', 'subset0')}")
        print("\n📝 O ingresa la ruta completa a LUNA16/subset0:")
        DATA_PATH = input("Ruta: ").strip().strip('"').strip("'")
        
        if not os.path.exists(DATA_PATH):
            raise FileNotFoundError(f"❌ La ruta no existe: {DATA_PATH}")
    
    # Buscar archivo de anotaciones
    luna16_root = os.path.dirname(DATA_PATH)  # Directorio LUNA16
    possible_annotation_paths = [
        os.path.join(luna16_root, 'annotations.csv'),
        os.path.join(DATA_PATH, 'annotations.csv'),
        os.path.join(project_root, 'LUNA16', 'annotations.csv'),
    ]
    
    ANNOTATIONS_PATH = None
    for path in possible_annotation_paths:
        if os.path.exists(path):
            ANNOTATIONS_PATH = path
            print(f"✅ Anotaciones encontradas en: {ANNOTATIONS_PATH}")
            break
    
    if ANNOTATIONS_PATH is None:
        print("⚠️  No se encontró annotations.csv (opcional)")

# Detectar automáticamente el primer archivo .mhd disponible
if os.path.exists(DATA_PATH):
    mhd_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.mhd')]
    if mhd_files:
        EXAMPLE_SCAN = mhd_files[0]
        print(f"\n📁 Configuración final:")
        print(f"  - Directorio de datos: {DATA_PATH}")
        print(f"  - Anotaciones: {ANNOTATIONS_PATH if ANNOTATIONS_PATH else 'No disponible'}")
        print(f"  - Archivos .mhd encontrados: {len(mhd_files)}")
        print(f"  - Archivo de ejemplo a usar: {EXAMPLE_SCAN}")
    else:
        raise FileNotFoundError(f"❌ No se encontraron archivos .mhd en {DATA_PATH}")
else:
    raise FileNotFoundError(f"❌ El directorio no existe: {DATA_PATH}")

---

# 📖 PARTE 1: Carga de Datos

## 1.1 Inicializar el Cargador de Datos

In [ ]:
# Crear instancia del cargador
loader = LUNA16DataLoader(DATA_PATH, ANNOTATIONS_PATH)

print("✅ LUNA16DataLoader inicializado")

## 1.2 Cargar un Escaneo CT

In [ ]:
# Ruta completa al archivo
scan_path = os.path.join(DATA_PATH, EXAMPLE_SCAN)

# Cargar imagen CT
print("📂 Cargando escaneo CT...")
ct_scan, origin, spacing = loader.load_itk_image(scan_path)

print("\n✅ Escaneo cargado correctamente")
print("\n📊 Información del volumen:")
print(f"  - Shape: {ct_scan.shape} (slices, height, width)")
print(f"  - Tipo de datos: {ct_scan.dtype}")
print(f"  - Tamaño en memoria: {ct_scan.nbytes / (1024**2):.1f} MB")
print(f"\n📏 Información espacial:")
print(f"  - Spacing: {spacing} mm (z, y, x)")
print(f"  - Origin: {origin} mm")
print(f"  - Dimensiones físicas: {ct_scan.shape * spacing} mm")
print(f"\n🔢 Rango de valores (Hounsfield Units):")
print(f"  - Mínimo: {ct_scan.min():.1f} HU")
print(f"  - Máximo: {ct_scan.max():.1f} HU")
print(f"  - Media: {ct_scan.mean():.1f} HU")
print(f"  - Desviación estándar: {ct_scan.std():.1f} HU")

## 1.3 Cargar Anotaciones de Nódulos

In [ ]:
# Extraer seriesuid del nombre del archivo
seriesuid = EXAMPLE_SCAN.replace('.mhd', '')

# Obtener anotaciones para este escaneo
annotations = loader.get_annotations_for_scan(seriesuid)

if annotations is not None and len(annotations) > 0:
    print(f"✅ Anotaciones cargadas: {len(annotations)} nódulos encontrados")
    print("\n📍 Nódulos anotados:")
    print(annotations[['coordX', 'coordY', 'coordZ', 'diameter_mm']])
    
    # Convertir a coordenadas voxel
    annotations_voxel = []
    for idx, row in annotations.iterrows():
        world_coords = np.array([row['coordZ'], row['coordY'], row['coordX']])
        voxel_coords = loader.world_to_voxel(world_coords, origin, spacing)
        annotations_voxel.append({
            'z': voxel_coords[0],
            'y': voxel_coords[1],
            'x': voxel_coords[2],
            'diameter': row['diameter_mm']
        })
    
    print(f"\n✅ Coordenadas convertidas a voxel")
else:
    print("ℹ️  No hay anotaciones para este escaneo")
    annotations_voxel = []

---

# 🔧 PARTE 2: Preprocesamiento

## 2.1 Inicializar Preprocesador

In [ ]:
# Crear instancia del preprocesador
preprocessor = LungPreprocessor()

print("✅ LungPreprocessor inicializado")
print("\n🛠️  Métodos disponibles:")
print("  - segment_lung_mask(): Segmentación pulmonar")
print("  - apply_clahe(): Realce de contraste adaptativo")
print("  - create_nodule_mask(): Crear máscara de nódulo 3D")

## 2.2 Seleccionar Slice de Trabajo

In [ ]:
# Seleccionar slice del medio del volumen
slice_idx = ct_scan.shape[0] // 2
ct_slice = ct_scan[slice_idx]

print(f"📍 Slice seleccionado: {slice_idx}/{ct_scan.shape[0]}")
print(f"  - Shape: {ct_slice.shape}")
print(f"  - Rango HU: [{ct_slice.min():.1f}, {ct_slice.max():.1f}]")

## 2.3 Segmentación Pulmonar

In [ ]:
# Segmentar pulmones
print("🔍 Segmentando región pulmonar...")
lung_mask = preprocessor.segment_lung_mask(ct_slice, threshold=-320)

# Calcular área pulmonar
area_pixels = np.sum(lung_mask)
area_mm2 = area_pixels * spacing[1] * spacing[2]
area_cm2 = area_mm2 / 100

print("\n✅ Segmentación completada")
print(f"  - Píxeles pulmonares: {area_pixels}")
print(f"  - Área pulmonar: {area_cm2:.1f} cm²")
print(f"  - Porcentaje del slice: {100 * area_pixels / ct_slice.size:.1f}%")

## 2.4 Normalización de Valores HU

In [ ]:
# Normalizar a rango [0, 1]
print("🔄 Normalizando valores Hounsfield...")
ct_normalized = loader.normalize_hu(ct_slice, min_hu=-1000, max_hu=400)

print("\n✅ Normalización completada")
print(f"  - Rango original: [{ct_slice.min():.1f}, {ct_slice.max():.1f}] HU")
print(f"  - Rango normalizado: [{ct_normalized.min():.3f}, {ct_normalized.max():.3f}]")

## 2.5 CLAHE (Opcional - para visualización mejorada)

In [ ]:
# Aplicar CLAHE para realzar contraste
print("✨ Aplicando CLAHE...")
ct_clahe = preprocessor.apply_clahe(ct_normalized, clip_limit=2.0, tile_size=(8, 8))

print("✅ CLAHE aplicado")
print("  (Útil para visualizar nódulos de baja densidad)")

---

# 📊 PARTE 3: Visualización de Resultados

## 3.1 Pipeline de Preprocesamiento Completo

In [ ]:
# Crear visualización del pipeline completo
visualizer = LungVisualizer()

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# 1. CT Original
axes[0].imshow(ct_slice, cmap='bone')
axes[0].set_title('1. CT Original\n(Unidades Hounsfield)', fontsize=12, weight='bold')
axes[0].axis('off')

# 2. Normalizado
axes[1].imshow(ct_normalized, cmap='bone')
axes[1].set_title('2. Normalizado [0,1]\n(Ventana pulmonar)', fontsize=12, weight='bold')
axes[1].axis('off')

# 3. Con CLAHE
axes[2].imshow(ct_clahe, cmap='bone')
axes[2].set_title('3. CLAHE\n(Contraste realzado)', fontsize=12, weight='bold')
axes[2].axis('off')

# 4. Máscara pulmonar
axes[3].imshow(lung_mask, cmap='gray')
axes[3].set_title(f'4. Máscara Pulmonar\nÁrea: {area_cm2:.0f} cm²', fontsize=12, weight='bold')
axes[3].axis('off')

# 5. Superposición
axes[4].imshow(ct_slice, cmap='bone')
axes[4].imshow(lung_mask, cmap='Reds', alpha=0.3)
axes[4].set_title('5. Segmentación\n(Superposición)', fontsize=12, weight='bold')
axes[4].axis('off')

# 6. Región pulmonar aislada
ct_lung_only = ct_normalized * lung_mask
axes[5].imshow(ct_lung_only, cmap='bone')
axes[5].set_title('6. Región Pulmonar\n(Solo pulmones)', fontsize=12, weight='bold')
axes[5].axis('off')

plt.suptitle(f'Pipeline de Preprocesamiento Completo - Slice {slice_idx}', fontsize=16, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("✅ Visualización del pipeline completada")

## 3.2 Visualización de Nódulos Anotados

In [ ]:
if len(annotations_voxel) > 0:
    print(f"📍 Visualizando {len(annotations_voxel)} nódulos anotados...\n")
    
    # Seleccionar el primer nódulo
    nodule = annotations_voxel[0]
    nodule_slice_idx = nodule['z']
    
    # Cargar slice que contiene el nódulo
    ct_slice_nodule = ct_scan[nodule_slice_idx]
    lung_mask_nodule = preprocessor.segment_lung_mask(ct_slice_nodule)
    
    # Preparar anotaciones para visualización
    slice_annotations = [{
        'x': nodule['x'],
        'y': nodule['y'],
        'diameter': nodule['diameter'] / spacing[1]  # Convertir mm a píxeles
    }]
    
    # Visualizar usando el módulo visualizer
    visualizer.plot_ct_with_annotations(
        ct_slice_nodule,
        lung_mask=lung_mask_nodule,
        annotations=slice_annotations,
        title=f"Nódulo de {nodule['diameter']:.1f}mm - Slice {nodule_slice_idx}",
        figsize=(18, 6)
    )
    
    print(f"✅ Nódulo visualizado en slice {nodule_slice_idx}")
    print(f"  - Coordenadas (voxel): Z={nodule['z']}, Y={nodule['y']}, X={nodule['x']}")
    print(f"  - Diámetro: {nodule['diameter']:.2f} mm")
    
else:
    print("ℹ️  No hay nódulos anotados en este escaneo para visualizar")

## 3.3 Visualización de Múltiples Slices del Volumen

In [ ]:
# Visualizar 9 slices distribuidos uniformemente
print("🎞️  Visualizando múltiples slices del volumen...\n")

visualizer.plot_volume_slices(
    ct_scan,
    num_slices=9,
    cmap='bone',
    title=f'Volumen CT Completo - {seriesuid[:30]}...'
)

print("✅ Visualización de volumen completada")

---

# 📈 PARTE 4: Análisis y Métricas

## 4.1 Análisis de Distribución de Valores HU

In [ ]:
# Analizar distribución de valores HU en región pulmonar
lung_hu_values = ct_slice[lung_mask == 1]

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histograma completo del slice
axes[0].hist(ct_slice.flatten(), bins=100, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(-1000, color='red', linestyle='--', linewidth=2, label='Aire (-1000 HU)')
axes[0].axvline(-320, color='orange', linestyle='--', linewidth=2, label='Threshold (-320 HU)')
axes[0].axvline(0, color='blue', linestyle='--', linewidth=2, label='Agua (0 HU)')
axes[0].set_xlabel('Unidades Hounsfield (HU)', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].set_title('Distribución HU - Slice Completo', fontsize=14, weight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Histograma solo región pulmonar
axes[1].hist(lung_hu_values, bins=50, color='lightcoral', alpha=0.7, edgecolor='black')
axes[1].axvline(lung_hu_values.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Media: {lung_hu_values.mean():.1f} HU')
axes[1].set_xlabel('Unidades Hounsfield (HU)', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('Distribución HU - Solo Región Pulmonar', fontsize=14, weight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Estadísticas de la región pulmonar:")
print(f"  - Media: {lung_hu_values.mean():.1f} HU")
print(f"  - Desviación estándar: {lung_hu_values.std():.1f} HU")
print(f"  - Mediana: {np.median(lung_hu_values):.1f} HU")
print(f"  - Percentil 25: {np.percentile(lung_hu_values, 25):.1f} HU")
print(f"  - Percentil 75: {np.percentile(lung_hu_values, 75):.1f} HU")

## 4.2 Evolución del Área Pulmonar a lo Largo del Volumen

In [ ]:
# Calcular área pulmonar en cada slice (muestreado)
print("📏 Calculando evolución del área pulmonar...\n")

lung_areas = []
slice_positions = []

for i in range(0, ct_scan.shape[0], 5):  # Cada 5 slices para velocidad
    lung_mask_temp = preprocessor.segment_lung_mask(ct_scan[i])
    area_mm2 = np.sum(lung_mask_temp) * spacing[1] * spacing[2]
    lung_areas.append(area_mm2 / 100)  # Convertir a cm²
    slice_positions.append(i)

# Graficar evolución
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(slice_positions, lung_areas, linewidth=2, color='steelblue', marker='o', markersize=4)
ax.fill_between(slice_positions, lung_areas, alpha=0.3, color='steelblue')

# Marcar máximo
max_area_idx = np.argmax(lung_areas)
max_slice = slice_positions[max_area_idx]
max_area = lung_areas[max_area_idx]
ax.axvline(max_slice, color='red', linestyle='--', alpha=0.7, linewidth=2)
ax.plot(max_slice, max_area, 'r*', markersize=20, label=f'Máx área: {max_area:.0f} cm² (slice {max_slice})')

ax.set_xlabel('Número de Slice', fontsize=12)
ax.set_ylabel('Área Pulmonar (cm²)', fontsize=12)
ax.set_title('Evolución del Área Pulmonar a lo Largo del Volumen CT', fontsize=14, weight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

plt.tight_layout()
plt.show()

print("✅ Análisis de área pulmonar completado")
print(f"\n📊 Resumen:")
print(f"  - Área máxima: {max_area:.1f} cm² (slice {max_slice})")
print(f"  - Área media: {np.mean(lung_areas):.1f} cm²")
print(f"  - Volumen pulmonar estimado: {np.sum(lung_areas) * spacing[0] / 10:.1f} cm³")

---

# 📝 PARTE 5: Resumen y Conclusiones

## 5.1 Resumen del Pipeline Ejecutado

In [ ]:
print("="*60)
print("📋 RESUMEN DEL PIPELINE COMPLETO")
print("="*60)

print("\n✅ PASOS EJECUTADOS:")
print("  1. ✓ Configuración del entorno (local/Colab)")
print("  2. ✓ Carga de datos LUNA16 (.mhd/.raw)")
print(f"  3. ✓ Escaneo cargado: {ct_scan.shape[0]} slices")
print(f"  4. ✓ Anotaciones: {len(annotations_voxel) if annotations_voxel else 0} nódulos")
print("  5. ✓ Segmentación pulmonar aplicada")
print("  6. ✓ Normalización HU [0,1]")
print("  7. ✓ CLAHE para realce de contraste")
print("  8. ✓ Visualizaciones generadas")
print("  9. ✓ Análisis de distribución HU")
print(" 10. ✓ Análisis de área pulmonar")

print("\n📊 DATOS PROCESADOS:")
print(f"  - Volumen: {ct_scan.shape}")
print(f"  - Tamaño: {ct_scan.nbytes / (1024**2):.1f} MB")
print(f"  - Spacing: {spacing} mm")
print(f"  - Nódulos encontrados: {len(annotations_voxel) if annotations_voxel else 0}")

print("\n🎯 RESULTADOS CLAVE:")
print(f"  - Área pulmonar (slice {slice_idx}): {area_cm2:.1f} cm²")
print(f"  - Volumen pulmonar estimado: {np.sum(lung_areas) * spacing[0] / 10:.1f} cm³")
print(f"  - HU medio región pulmonar: {lung_hu_values.mean():.1f} HU")

print("\n📚 NOTEBOOKS RELACIONADOS:")
print("  - 01_preprocesamiento.ipynb: Experimentación con preprocesamiento")
print("  - 02_visualizacion.ipynb: Técnicas avanzadas de visualización")

print("\n" + "="*60)
print("✅ PIPELINE COMPLETADO EXITOSAMENTE")
print("="*60)

## 5.2 Próximos Pasos Sugeridos

### Para profundizar en cada etapa:

1. **Preprocesamiento detallado**: Abre `01_preprocesamiento.ipynb`
   - Experimenta con diferentes valores de threshold
   - Comprende el algoritmo de `clear_border()` paso a paso
   - Prueba diferentes parámetros de CLAHE

2. **Visualización avanzada**: Abre `02_visualizacion.ipynb`
   - Rendering 3D de superficies pulmonares
   - Visualización interactiva con sliders
   - Comparación NDCT vs LDCT
   - Mapas de calor de densidad

3. **Próximos notebooks** (en desarrollo):
   - `03_segmentacion_nodulos.ipynb`: Detección automática de nódulos
   - `04_clasificacion_nodulos.ipynb`: Clasificación benigno/maligno
   - `05_entrenamiento_modelo.ipynb`: Entrenar redes neuronales

### Para experimentar con tus propios datos:

1. Ajusta las rutas en la sección "Configuración de Rutas"
2. Cambia `EXAMPLE_SCAN` por el archivo que quieras analizar
3. Re-ejecuta el notebook completo

### Para usar el código en tus propios scripts:

```python
from utils import LUNA16DataLoader, LungPreprocessor, LungVisualizer

# Tu código aquí
loader = LUNA16DataLoader(data_path, annotations_path)
preprocessor = LungPreprocessor()
# ...
```

---

**¿Preguntas?** Consulta la documentación en [`docs/`](../docs/) o revisa el [`README.md`](../README.md) principal.